## MSDS 432 - Programming Assignment #8 Dan Avni

The purpose of this programming assignment is the implement a greedy algorithm to solve a simple scheduling problem. Greedy algorithms are one that seek the best local solution at each step of the algorithm. They often fail to find a global optimum. Some special cases of greedy algorithm can be shown to find the global optimum. The Dijkstra’s shortest path algorithm is such case. In other cases, the traveling salesperson for example, it may be cost prohibitive to seek a global optimum, so a greedy algorithm can identify a reasonable heuristic solution.

Our toy example seeks to schedule security guards to cover a 24x7 coverage of a construction site. We have a pool of six guards and try to avoid paying overtime by limiting each guard to an 8-hour shift in any 24-hour period. We solve the problem via a greedy algorithm, identifying the security guard with the longest time from his/hers last shift. To simplify, we will raise an exception is no solution can be identified. If a solution is found, it is ought to be the optimal one, since no overtime payment are due, and the total weekly cost will be:

$Cost = 24hrs * 7days * \$15 = \$2,520$

The solution can be east extended to cover a rolling 24*7 scheduling. As far as runtime performance, we expect a runtime complexity of $O(n^2)$ with $n$ representing the number of available guards. This is due to ineffient data structure. We could improve the runtime performance to $O(nlog(n))$ by using a min-priority queue.

The below table shows the solution arranged by day and shift, including the guard and time since last shift, showing how the constraints were satisfied and the minimal wage should be paid.

| Day   | Night   | Morning   | Evening  |   ('Time Since Last', 'Night') |   ('Time Since Last', 'Morning') |   ('Time Since Last', 'Evening') |
|:------|:---------------------|:-----------------------|:-----------------------|-------------------------------:|---------------------------------:|---------------------------------:|
| Mon   | Dasher               | Dancer                 | Prancer                |                            inf |                              inf |                              inf |
| Tue   | Comet                | Cupid                  | Dunder                 |                            inf |                              inf |                              inf |
| Wed   | Dasher               | Dancer                 | Prancer                |                             48 |                               48 |                               48 |
| Thu   | Comet                | Cupid                  | Dunder                 |                             48 |                               48 |                               48 |
| Fri   | Dasher               | Dancer                 | Prancer                |                             48 |                               48 |                               48 |
| Sat   | Comet                | Cupid                  | Dunder                 |                             48 |                               48 |                               48 |
| Sun   | Dasher               | Dancer                 | Prancer                |                             48 |                               48 |                               48 |

In [13]:
#import required packages
import pandas as pd
from tabulate import tabulate

In [14]:
#greedy method to return the next best guard for the offset
def get_next_shift(guards, offset):
    best_time  = 0
    best_guard = None
    
    for guard in guards:
        current = offset - guards[guard] 
        if current > best_time:
            best_time  = current
            best_guard = guard 
    
    guards[best_guard] = offset
    
    return best_guard, best_time

In [15]:
#initialize guard data
def init_data():
    Inf  = float("inf")
    guards =  \
        {
            'Dasher': -Inf, 'Dancer': -Inf, 'Prancer': -Inf, 
            'Comet': -Inf,  'Cupid':  -Inf, 'Dunder':  -Inf
        }
    return guards

In [16]:
#run greedy algorithm to populate the weekly schedule
schedule_cols = ['Day', 'Shift', 'Guard', 'Time Since Last']
schedule_df   = pd.DataFrame(columns = schedule_cols) 

guards = init_data()

offset = 0
for day in ['Mon', 'Tue', 'Wed', 'Thu', 'Fri', 'Sat', 'Sun']:
    for shift in ['Night', 'Morning', 'Evening']:
        guard, last_shift = get_next_shift(guards, offset)
        schedule_df.loc[len(schedule_df)] = [day, shift, guard, last_shift]
        if last_shift < 24:
            raise Exception("No solution found!")
        offset += 8

In [17]:
#output schedule
schedule_df

,Day,Shift,Guard,Time Since Last
0,Mon,Night,Dasher,inf
1,Mon,Morning,Dancer,inf
2,Mon,Evening,Prancer,inf
3,Tue,Night,Comet,inf
4,Tue,Morning,Cupid,inf
5,Tue,Evening,Dunder,inf
6,Wed,Night,Dasher,48.000000
7,Wed,Morning,Dancer,48.000000
8,Wed,Evening,Prancer,48.000000
9,Thu,Night,Comet,48.000000


In [18]:
#transform schedule to hourly and sort by day
days = ['Mon','Tue','Wed','Thu','Fri','Sat', 'Sun']
schedule_df['Day'] = pd.Categorical(schedule_df['Day'], categories=days, ordered=True)
shifts = ['Night','Morning','Evening']
schedule_df['Shift'] = pd.Categorical(schedule_df['Shift'], categories=shifts, ordered=True)

daily_schedule_df = schedule_df.pivot(index='Day', columns='Shift')

In [19]:
daily_schedule_df

Guard                  Time Since Last                      
Shift   Night Morning  Evening           Night    Morning    Evening
Day                                                                 
Mon    Dasher  Dancer  Prancer             inf        inf        inf
Tue     Comet   Cupid   Dunder             inf        inf        inf
Wed    Dasher  Dancer  Prancer       48.000000  48.000000  48.000000
Thu     Comet   Cupid   Dunder       48.000000  48.000000  48.000000
Fri    Dasher  Dancer  Prancer       48.000000  48.000000  48.000000
Sat     Comet   Cupid   Dunder       48.000000  48.000000  48.000000
Sun    Dasher  Dancer  Prancer       48.000000  48.000000  48.000000

In [20]:
#output the daily schedule in a markdown friendly format
print(tabulate(daily_schedule_df, tablefmt="pipe", headers="keys"))

| Day   | ('Guard', 'Night')   | ('Guard', 'Morning')   | ('Guard', 'Evening')   |   ('Time Since Last', 'Night') |   ('Time Since Last', 'Morning') |   ('Time Since Last', 'Evening') |
|:------|:---------------------|:-----------------------|:-----------------------|-------------------------------:|---------------------------------:|---------------------------------:|
| Mon   | Dasher               | Dancer                 | Prancer                |                            inf |                              inf |                              inf |
| Tue   | Comet                | Cupid                  | Dunder                 |                            inf |                              inf |                              inf |
| Wed   | Dasher               | Dancer                 | Prancer                |                             48 |                               48 |                               48 |
| Thu   | Comet                | Cupid                  | Dunder      